In [1]:
import os
import shutil
from tqdm import tqdm
import json
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split


In [2]:
# 개인 로컬마다 설정 해야하는 부분 #


# cctv_df 가 있는 위치
path_cctv_df = 'C:\\Users\\user\\section6\\tp2\\data'

# 이미지 파일 한 곳에 모아둔 위치
path_cctv_image = 'C:\\Users\\user\\section6\\tp2\\data\\precleaning_data\\images'

# cctv 이미지만 모아둘 위치 
new_path_cctv_image = 'C:\\Users\\user\\section6\\tp2\\code_file\\datasets\\images'

# 라벨 파일 한 곳에 모아둔 위치
path_cctv_label = 'C:\\Users\\user\\section6\\tp2\\data\\precleaning_data\\labels'

# cctv 라벨 파일 한 곳에 모아둘 위치
new_path_cctv_label = 'C:\\Users\\user\\section6\\tp2\\code_file\\datasets\\labels'

In [3]:
df = pd.read_csv(path_cctv_df + '/cctv_df.csv')

In [4]:
# video_id의 유니크 값만 추출
video_id = df['video_id'].unique()

In [5]:
# 이미지 파일 개수 26,671개
len(video_id)

26671

# CCTV 이미지, Json만 새로운 폴더로 이동 

In [23]:
# cctv 이미지만 새로운 폴더로 카피
for video in tqdm(video_id):
    shutil.copy(path_cctv_image + '/' + video + '.jpg',
               new_path_cctv_image + '/' + video + '.jpg')
    
# 26671 개 데이터가 잘 들어갔는지 확인

  3%|██▏                                                                          | 761/26671 [00:03<01:59, 217.47it/s]


KeyboardInterrupt: 

In [21]:
# cctv label 데이터만 json -> txt

for video in tqdm(video_id):
    with open(path_cctv_label + '/' + video + '.json', encoding = 'utf-8') as f:
        data = json.load(f)
        
    # new_path에 txt 파일 형식으로 저장 
    with open(new_path_cctv_label + '/' + video + '.txt', 'w+') as f_txt:
        
        for i in range(len(data['annotations']['PM'])):
            pm_code = int(data['annotations']['PM'][i]['PM_code']) - 13
            if pm_code >= 0 and pm_code <= 6:
                width = data['annotations']['PM'][i]['points'][2] 
                height = data['annotations']['PM'][i]['points'][3]
                x = data['annotations']['PM'][i]['points'][0] + width / 2
                y = data['annotations']['PM'][i]['points'][1] + height / 2  
                new_line = f'{pm_code} {x} {y} {width} {height}'

                f_txt.write(new_line)
                f_txt.write('\n')
        
# 26671 개 데이터가 잘 들어갔는지 확인 

100%|███████████████████████████████████████████████████████████████████████████| 26671/26671 [02:46<00:00, 160.36it/s]


# Train, Val, Test 80:15:5 비율로 나누기

In [6]:
# video_id에서 train, test 나눔
train, test = train_test_split(video_id, 
                               test_size=0.2, 
                               random_state=42)

In [7]:
train[0], test[0]

('F000074_000_5752_C_D_F_1', 'S002170_010_0252_C_D_F_0')

In [8]:
len(train), len(test)

(21336, 5335)

In [9]:
# test에서 val, test 나눔
val, test = train_test_split(test,
                            test_size=0.25,
                            random_state=42)

In [10]:
# 최종 80 : 15 : 5 비율로 나눔 
len(train), len(val), len(test)

(21336, 4001, 1334)

# image, label 데이터 Train, val, test 폴더에 맞게 나누기 

In [27]:
# image 파일 train, val, test 폴더로 나눠서 저장
for video in tqdm(video_id):
    if video in train:
        shutil.move(new_path_cctv_image + '/' + video +'.jpg',
                   new_path_cctv_image + '/train/' + video + '.jpg')
    elif video in val:
        shutil.move(new_path_cctv_image + '/' + video +'.jpg',
                   new_path_cctv_image + '/val/' + video + '.jpg')
    elif video in test:
        shutil.move(new_path_cctv_image + '/' + video +'.jpg',
                   new_path_cctv_image + '/test/' + video + '.jpg')
    else:
        # 있지는 않지만 혹시 모르니 프린트
        print(video)

100%|███████████████████████████████████████████████████████████████████████████| 26671/26671 [00:28<00:00, 921.56it/s]


In [12]:
# label 파일 train, val, test 폴더로 나눠서 저장
for video in tqdm(video_id):
    if video in train:
        shutil.move(new_path_cctv_label + '/' + video +'.txt',
                   new_path_cctv_label + '/train/' + video + '.txt')
    elif video in val:
        shutil.move(new_path_cctv_label + '/' + video +'.txt',
                   new_path_cctv_label + '/val/' + video + '.txt')
    elif video in test:
        shutil.move(new_path_cctv_label + '/' + video +'.txt',
                   new_path_cctv_label + '/test/' + video + '.txt')
    else:
        # 있지는 않지만 혹시 모르니 프린트
        print(video)

  0%|                                                  | 0/26671 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\section6\\tp2\\code_file\\datasets\\labels/C000002_003_0009_C_D_F_0.txt'

# 해상도 변경 후 각 해상도 변경 폴더에 넣기

In [11]:
# 이미지 해상도 변경 폴더 위치
path_640_image = 'C:\\Users\\user\\section6\\tp2\\code_file\\datasets_640\\images'

# 라벨 해상도 변경 폴더 위치
path_640_label = 'C:\\Users\\user\\section6\\tp2\\code_file\\datasets_640\\labels'

In [25]:
# 이미지 해상도 변경
for video in tqdm(video_id):
    img = cv2.imread(path_cctv_image + '/' + video + '.jpg')
    img_resize = cv2.resize(img, dsize=(640, 640), interpolation=cv2.INTER_AREA)
    """
    cv2.resize 파리미터 
    src: 원본 이미지
    dsize: 해상도. (가로,세로) 형태로 값을 지정. 꼭 소괄호 형태로 지정.
    fx: 가로 확대.축소 비율 설정 0.5면 가로가 절반으로 줄어듬
    fy: 세로 확대.축소 비율 설정
    interpolation : 사전적 의미로는 보간법이라고 하는데 이미지 크기가 바뀌면서 생기는 문제를 해결하기 위한 방법
        cv2.INTER_NEAREST : 인접한 픽셀들의 정보 이용
        cv2.INTER_LINEAR : 쌍선형 보간법 이용
        cv2.INTER_AREA : pixel area relation을 이용하여 resampling
        cv2.INTER_CUBIC: 4x4 픽셀 이웃들을 이용
        cv2.INTER_LANCZOS4: 8x8 픽셀 이웃들을 이용
    """
    if video in train:
        cv2.imwrite(path_640_image + '/train/' + video + '.jpg', img_resize)
    elif video in val:
        cv2.imwrite(path_640_image + '/val/' + video + '.jpg', img_resize)
    elif video in test:
        cv2.imwrite(path_640_image + '/test/' + video + '.jpg', img_resize)
    else:
        # 있지는 않지만 혹시 모르니 프린트
        print(video)
        
    # 약 15 ~ 20 분 소요    
    # train : 21336, val : 4001, test : 1334 데이터 개수 확인 

100%|████████████████████████████████████████████████████████████████████████████| 26671/26671 [18:55<00:00, 23.50it/s]


In [12]:
def change_label_640(j_data, v_name, cate, path=path_640_label):
    """
    change_label_640 : 라벨링 데이터 수정 (0~1로 정규화 진행) 
                       및 txt 파일로 저장
    j_data : json file,
    v_name : video_name
    cate : train, val, test 종류,
    path : 640 라벨 데이터 위치
    """
    with open(path + '/' + cate + '/' + v_name + '.txt', "w+") as f_txt:
        
        for i in range(len(j_data['annotations']['PM'])):
            pm_code = int(j_data['annotations']['PM'][i]['PM_code']) - 13
            if pm_code >= 0 and pm_code <= 6:
                # 정규화 코드 
                width = j_data['annotations']['PM'][i]['points'][2] / 1920
                height = j_data['annotations']['PM'][i]['points'][3] / 1080
                x = (j_data['annotations']['PM'][i]['points'][0] + j_data['annotations']['PM'][i]['points'][2]/2) / 1920
                y = (j_data['annotations']['PM'][i]['points'][1] + j_data['annotations']['PM'][i]['points'][3]/2) / 1080  
                new_line = f'{pm_code} {x} {y} {width} {height}'
                
                f_txt.write(new_line)
                f_txt.write('\n')

In [13]:
# cctv label 데이터 640 형식으로 수정
for video in tqdm(video_id):
    with open(path_cctv_label + '/' + video + '.json', encoding = 'utf-8') as f:
        data = json.load(f)

    # new_path에 txt 파일 형식으로 저장 
    if video in train:
        change_label_640(j_data=data, v_name=video, cate='train')
    elif video in val:
        change_label_640(j_data=data, v_name=video, cate='val')
    elif video in test:
        change_label_640(j_data=data, v_name=video, cate='test')
    else:
        print(video)
        
    # train : 21336, val : 4001, test : 1334 데이터 개수 확인 

100%|████████████████████████████████████████████████████████████████████████████| 26671/26671 [05:30<00:00, 80.75it/s]


# 흑백 이미지 변경 후 흑백 폴더에 넣기

In [12]:
# 흑백 이미지 변경 폴더 위치
path_black = 'C:\\Users\\user\\section6\\tp2\\code_file\\datasets_640_black'
path_black_image = path_black + '\\images'
path_black_labels = path_black + '\\labels'

In [18]:
for video in tqdm(video_id):
    # gray scale로 불러오기
    img = cv2.imread(path_cctv_image + '/' + video + '.jpg', 
                     cv2.IMREAD_GRAYSCALE)
    # 사이즈 재조정 
    img_resize = cv2.resize(img, dsize=(640, 640), 
                            interpolation=cv2.INTER_AREA)
    
    if video in train:
        cv2.imwrite(path_black_image + '/train/' + video + '.jpg', img_resize)
    elif video in val:
        cv2.imwrite(path_black_image + '/val/' + video + '.jpg', img_resize)
    elif video in test:
        cv2.imwrite(path_black_image + '/test/' + video + '.jpg', img_resize)
    else:
        # 있지는 않지만 혹시 모르니 프린트
        print(video)

100%|████████████████████████████████████████████████████████████████████████████| 26671/26671 [14:31<00:00, 30.62it/s]


In [15]:
# Test

test_video_id = video_id[120]

test_img = cv2.imread(path_cctv_image + '/' + test_video_id + '.jpg')
test_img = cv2.resize(test_img, dsize=(640, 640),
                     interpolation=cv2.INTER_AREA)

with open(path_cctv_label + '/' + test_video_id + '.json', encoding = 'utf-8') as f:
    data = json.load(f)
    
    for i in range(len(data['annotations']['PM'])):
        pm_code = int(data['annotations']['PM'][i]['PM_code']) - 13
        if pm_code >= 0 and pm_code <= 6:
            # 정규화 코드 
            width = data['annotations']['PM'][i]['points'][2] / 1920
            height = data['annotations']['PM'][i]['points'][3] / 1080
            x = (data['annotations']['PM'][i]['points'][0] + data['annotations']['PM'][i]['points'][2]/2) / 1920
            y = (data['annotations']['PM'][i]['points'][1] + data['annotations']['PM'][i]['points'][3]/2) / 1080
            

print(width, height, x, y)
pt1 = (int((x - width/2)*640), int((y - height/2)*640))
pt2 = (int((x + width/2)*640), int((y + height/2)*640))

print(pt1, pt2)
red_colr = (0,0,255)

test_img = cv2.rectangle(test_img, pt1, pt2, red_colr, 3)

cv2.imshow('retanle', test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.11353190636435487 0.14467640232388368 0.19881898263070003 0.5663092663172345
(90, 316) (163, 408)
